<a href="https://colab.research.google.com/github/JoybirSingh/NLP-MINI-PROJECTS/blob/main/assign4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_article(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'lxml')
        article = soup.find('article')
        if article:
            paragraphs = article.find_all('p')
            return '\n'.join(p.text for p in paragraphs)
    return None

# for hindi
def scrape_article_hindi(url):
  response = requests.get(url)
  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'lxml')
    article = soup.find('main')
    if article:
      paragraphs = article.find_all('p')
      return '\n'.join(p.text for p in paragraphs)
  return None

# Example URLs for BBC News in Hindi and English
hindi_url = 'https://www.bbc.com/hindi/articles/c4nny99n692o'
english_url = 'https://www.bbc.com/news/articles/c0kkyvq081wo'

hindi_content = scrape_article_hindi(hindi_url)
english_content = scrape_article(english_url)

print("Hindi Content:")
print(hindi_content[:500])  # Print first 500 characters

print("\nEnglish Content:")
print(english_content[:500])  # Print first 500 characters


Hindi Content:
इमेज स्रोत, ANI 
पश्चिम बंगाल के दार्जिलिंग में हुए एक रेल हादसे में 9 लोगों की मौत हो गई है. इस हादसे में 46 लोग घायल भी हुए हैं.
नॉर्थ फ्रंटियर रेलवे के प्रवक्ता ने बीबीसी हिंदी को बताया, "हादसे में अब तक कुल 9 लोगों की मौत हुई है, जबकि 46 लोग घायल हुए हैं. हादसे की वजह से इलाके में 8 ट्रेनें रद्द हुई हैं, जबकि 24 ट्रेनों के रूट बदले गए हैं."
रेलवे के मुताबिक मरने वालों में तीन रेलवे कर्मचारी हैं. 
यह हादसा सोमवार सुबह क़रीब पौने 9 बजे हुआ है.
जिस वक़्त यह हादसा हुआ उस वक़्त कंचनजंघा एक्सप्रेस

English Content:
Eight people have died and around 60 have been injured in a train crash in the eastern Indian state of West Bengal.
The accident occurred when a goods train hit a stationary passenger train, the Kanchenjungha Express, in the New Jalpaiguri area on Monday morning.
Dramatic visuals from the accident site showed one wagon of the express train suspended in the air.
Authorities say "human error" may have led to the crash and a detailed investigation will be held. 
Of

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Function to split text into smaller chunks
def chunk_text(text, max_tokens, tokenizer):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding="longest")["input_ids"][0]
    chunks = []
    while len(tokens) > max_tokens:
        chunks.append(tokens[:max_tokens])
        tokens = tokens[max_tokens:]
    chunks.append(tokens)
    return chunks

# Translation function for long texts
def translate_long_text(text, tokenizer, model, max_tokens=512):
    chunks = chunk_text(text, max_tokens, tokenizer)
    translated_chunks = [model.generate(chunk.unsqueeze(0)) for chunk in chunks]
    translated_texts = [tokenizer.decode(t[0], skip_special_tokens=True) for t in translated_chunks]
    return " ".join(translated_texts)

# Translation model setup
model_name = 'Helsinki-NLP/opus-mt-hi-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Translating long text
translated_content = translate_long_text(hindi_content, tokenizer, model)

print("Translated Content:")
print(translated_content[:500])  # Print first 500 characters


Translated Content:
I've got 9 people killed in a traffic accident in Darga, in the wild country of ANIZA, and 46 people are injured in this accident. The government of the North African railway said, "A total of 9 people have died in the country, while the disaster has been canceled by a group of 46 people because of the accident.


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def evaluate_translation(reference, translation):
    reference_tokens = reference.split()
    translation_tokens = translation.split()
    score = sentence_bleu([reference_tokens], translation_tokens)
    return score

# Assuming we have the original English content
reference = english_content
translation = translated_content

bleu_score = evaluate_translation(reference, translation)
print("BLEU Score:", bleu_score)


BLEU Score: 2.171057086056909e-81
